In [1]:
import os
import sys
from pathlib import Path
django_project_dir = Path(".")
sys.path.insert(0, str(django_project_dir))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "newsilkroad.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

[raven.contrib.django.client.DjangoClient]DEBUG 15:55:58 Configuring Raven for host: None
[raven.contrib.django.client.DjangoClient]INFO 15:55:58 Raven is not configured (logging is disabled). Please see the documentation for more information.


In [27]:
from writings.models import Entry, EntryCollection, Category
from django.db.models import Q
import csv

In [28]:
def _get_categories(obj):
    if obj.categories:
        return ",".join([x.name for x in obj.categories.all()])
    return ""

def _get_tags(obj):
    if obj.tags:
        return ",".join([x.name for x in obj.tags.all()])
    return ""

def _get_related_entries(obj):
    if obj.related_entries:
        return ",".join([x.title for x in obj.related_entries.all()])
    return ""

def _get_filer_info(obj):
    value = ''
    if obj.featured_image:
        value = f"{getattr(obj.featured_image, 'original_filename')},{getattr(obj.featured_image, 'default_caption')}"
    return value

In [29]:
entry_fields =  {
    'id': '',
    'created_at': '',
    'updated_at': '',
    'published': '',
    'title': '',
    'subtitle': '',
    'slug': '',
    'author': '',
    'content': '',
    'content_rendered': '',
    'description': '',
    'description_rendered': '',
    'share_text': '',
    'featured_image_id': _get_filer_info,
    'publication_date': '',
    'categories': _get_categories,
    'tags': _get_tags,
    'related_entries': _get_related_entries,
}

In [78]:
e = Entry.objects.filter(published=True).exclude(Q(categories__slug="news") & Q(author__exact=""))
e.count()

318

In [9]:
Entry.objects.all().count()

1283

In [79]:
with open('entries.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=entry_fields.keys(), quoting=csv.QUOTE_MINIMAL, quotechar='"')
    writer.writeheader()
    for e in Entry.objects.filter(published=True).exclude(Q(categories__slug="news") & Q(author__exact="")).prefetch_related():
        row = entry_fields.copy()
        for attr in entry_fields.keys():
            if row[attr] == '':
                row[attr] = getattr(e, attr)
                continue
            row[attr] = row[attr](e)
        writer.writerow(row)